In [ ]:
!pip install -q xgboost
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
import boto3
from sagemaker import get_execution_role
import sagemaker

# Use SageMaker
SageMaker = True

# Use back-test files
back_test = False
if back_test:
  bt_frag = '-back'
else:
  bt_frag = ''

# Set scaled boolean
sclaled_on = True
# Set XGBoost params 
full_params = True

if not SageMaker:
    from google.colab import drive
    drive.mount('/content/drive')
    #!ls "/content/drive/My Drive/LSTM Futures"
    #!cp '/content/drive/My Drive/LSTM Futures/Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv' 'Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv'
    #!cp '/content/drive/My Drive/LSTM Futures/daily-granular' + bt_frag + '.csv' 'daily-granular' + bt_frag + '.csv'
    EminiSP = pd.read_csv('/content/drive/My Drive/LSTM Futures/Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv')
    EminiSPpredict = pd.read_csv('/content/drive/My Drive/LSTM Futures/daily-granular' + bt_frag + '.csv')
else:
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    bucket='eminisp500vbloise'
    train_data_key = 'Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv'
    inference_data_key = 'daily-granular' + bt_frag + '.csv'
    train_data_location = 's3://{}/{}'.format(bucket, train_data_key)
    inference_data_location = 's3://{}/{}'.format(bucket, inference_data_key)
    EminiSP = pd.read_csv(train_data_location)
    EminiSPpredict = pd.read_csv(inference_data_location)

X = EminiSP.copy()
X.drop(['date-1', 'time-1', 'time-2', 'time-3', 'time-4', 'time-5', 'outcome', 'b_outcome'], axis=1, inplace=True) #,'percent-change-1', 'percent-change-2', 'percent-change-3'
Y = EminiSP.pop('b_outcome')
EminiSPpredict.drop(['date-1', 'time-1', 'time-2', 'time-3', 'time-4', 'time-5'], axis=1, inplace=True) #,'percent-change-1', 'percent-change-2', 'percent-change-3'

print(X.head())
print(X.shape)

print(Y.head())
print(Y.shape)

print(EminiSPpredict.head())
print(EminiSPpredict.shape)

# split data into X and y
#X = dataset[:,0:25]
#Y = dataset[:,25]
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# Scaling the data
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoder.fit(y_test)
encoded_y_test = encoder.transform(y_test)
from sklearn.preprocessing import MinMaxScaler

Target_scaler = MinMaxScaler(feature_range=(0, 1))
Feature_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = Feature_scaler.fit_transform(np.array(X_train))
X_test_scaled = Feature_scaler.fit_transform(np.array(X_test))

y_train_scaled = Target_scaler.fit_transform(np.array(encoded_y_train).reshape(-1,1))
y_test_scaled = Target_scaler.fit_transform(np.array(encoded_y_test).reshape(-1,1))
EminiSPpredict_scaled = Feature_scaler.fit_transform(np.array(EminiSPpredict))

# fit model no training data
# Defaults
# XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
#       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
#       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#       scale_pos_weight=1, seed=0, silent=True, subsample=1)
if full_params:
  model = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.05,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=1, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, subsample=1, use_label_encoder=False, eval_metric='logloss')
else:
  model = XGBClassifier()
if not sclaled_on:
  model.fit(X_train, encoded_y_train)
  # make predictions for test data
  y_pred = model.predict(X_test)
else:
  model.fit(X_train_scaled, encoded_y_train)
  # make predictions for test data
  y_pred = model.predict(X_test_scaled)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(encoded_y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
if not sclaled_on:  
  # Predict the model
  pred = model.predict(EminiSPpredict)
  print("Prediction : % f" %(pred))
  # RMSE Computation
  #rmse = np.sqrt(MSE(y_test, pred))
  #print("RMSE : % f" %(rmse))
else:
  # scaled
  pred = model.predict(EminiSPpredict_scaled)
  pred_rescaled = Target_scaler.inverse_transform(pred.reshape(1,-1))
  print("Prediction : % f" %(pred_rescaled))
  # RMSE Computation
  #rmse = np.sqrt(MSE(y_test_scaled, pred))
  #print("RMSE : % f" %(rmse))

[autoreload of IPython.utils.ipstruct failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    if do_reload:
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    # IPython connectivity
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_property),
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    old_obj = getattr(old, key)
  File "/usr/local/lib/python3.9/site-packages/IPython/utils/ipstruct.py", line 117, in __setattr__
    raise AttributeError(
AttributeError: attr __class__ is a protected member of class Struct.
]
[autoreload of IPython.core.display failed: Traceback (most recent call last):
  Fil

In [ ]:
!pip install -q xgboost
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
import boto3
from sagemaker import get_execution_role
import sagemaker

# Use SageMaker
SageMaker = True

# Use back-test files
back_test = False
if back_test:
  bt_frag = '-back'
else:
  bt_frag = ''

# Set scaled boolean
sclaled_on = True
# Set XGBoost params 
full_params = True

if not SageMaker:
    from google.colab import drive
    drive.mount('/content/drive')
    #!ls "/content/drive/My Drive/LSTM Futures"
    #!cp '/content/drive/My Drive/LSTM Futures/Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv' 'Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv'
    #!cp '/content/drive/My Drive/LSTM Futures/daily-granular' + bt_frag + '.csv' 'daily-granular' + bt_frag + '.csv'
    EminiSP = pd.read_csv('/content/drive/My Drive/LSTM Futures/Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv')
    EminiSPpredict = pd.read_csv('/content/drive/My Drive/LSTM Futures/daily-granular' + bt_frag + '.csv')
else:
    sagemaker_session = sagemaker.Session()
    role = get_execution_role()
    bucket='eminisp500vbloise'
    train_data_key = 'Micro-Emini-SP500-MES-F-granular' + bt_frag + '.csv'
    inference_data_key = 'daily-granular' + bt_frag + '.csv'
    train_data_location = 's3://{}/{}'.format(bucket, train_data_key)
    inference_data_location = 's3://{}/{}'.format(bucket, inference_data_key)
    EminiSP = pd.read_csv(train_data_location)
    EminiSPpredict = pd.read_csv(inference_data_location)

X = EminiSP.copy()
X.drop(['date-1', 'time-1', 'time-2', 'time-3', 'time-4', 'time-5', 'outcome', 'b_outcome'], axis=1, inplace=True) #,'percent-change-1', 'percent-change-2', 'percent-change-3'
Y = EminiSP.pop('b_outcome')
EminiSPpredict.drop(['date-1', 'time-1', 'time-2', 'time-3', 'time-4', 'time-5'], axis=1, inplace=True) #,'percent-change-1', 'percent-change-2', 'percent-change-3'

print(X.head())
print(X.shape)

print(Y.head())
print(Y.shape)

print(EminiSPpredict.head())
print(EminiSPpredict.shape)

# split data into X and y
#X = dataset[:,0:25]
#Y = dataset[:,25]
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# Scaling the data
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)
encoder.fit(y_train)
encoded_y_train = encoder.transform(y_train)
encoder.fit(y_test)
encoded_y_test = encoder.transform(y_test)
from sklearn.preprocessing import MinMaxScaler

Target_scaler = MinMaxScaler(feature_range=(0, 1))
Feature_scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = Feature_scaler.fit_transform(np.array(X_train))
X_test_scaled = Feature_scaler.fit_transform(np.array(X_test))

y_train_scaled = Target_scaler.fit_transform(np.array(encoded_y_train).reshape(-1,1))
y_test_scaled = Target_scaler.fit_transform(np.array(encoded_y_test).reshape(-1,1))
EminiSPpredict_scaled = Feature_scaler.fit_transform(np.array(EminiSPpredict))

# fit model no training data
# Defaults
# XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
#       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
#       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#       scale_pos_weight=1, seed=0, silent=True, subsample=1)
if full_params:
  model = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.05,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=1, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, subsample=1, use_label_encoder=False, eval_metric='logloss')
else:
  model = XGBClassifier()
if not sclaled_on:
  model.fit(X_train, encoded_y_train)
  # make predictions for test data
  y_pred = model.predict(X_test)
else:
  model.fit(X_train_scaled, encoded_y_train)
  # make predictions for test data
  y_pred = model.predict(X_test_scaled)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(encoded_y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
if not sclaled_on:  
  # Predict the model
  pred = model.predict(EminiSPpredict)
  print("Prediction : % f" %(pred))
  # RMSE Computation
  #rmse = np.sqrt(MSE(y_test, pred))
  #print("RMSE : % f" %(rmse))
else:
  # scaled
  pred = model.predict(EminiSPpredict_scaled)
  pred_rescaled = Target_scaler.inverse_transform(pred.reshape(1,-1))
  print("Prediction : % f" %(pred_rescaled))
  # RMSE Computation
  #rmse = np.sqrt(MSE(y_test_scaled, pred))
  #print("RMSE : % f" %(rmse))

[autoreload of IPython.utils.ipstruct failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    if do_reload:
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    # IPython connectivity
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_property),
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    old_obj = getattr(old, key)
  File "/usr/local/lib/python3.9/site-packages/IPython/utils/ipstruct.py", line 117, in __setattr__
    raise AttributeError(
AttributeError: attr __class__ is a protected member of class Struct.
]
[autoreload of IPython.core.display failed: Traceback (most recent call last):
  Fil